Using device: cuda
Dataset path: Driver Drowsiness Dataset (DDD)
Model will be saved to: drowsiness_model.pth
Loading dataset...
Loading drowsy images from: Driver Drowsiness Dataset (DDD)\Drowsy
Loading non-drowsy images from: Driver Drowsiness Dataset (DDD)\Non Drowsy
Total images: 41793, Drowsy: 22348, Non-Drowsy: 19445
Creating data loaders...
Creating model...


c:\ProgramData\anaconda3\envs\gpuenv2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\envs\gpuenv2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Ishon Panda/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:03<00:00, 6.06MB/s]
c:\ProgramData\anaconda3\envs\gpuenv2\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarn

Starting training...


Epoch 1/15 [Val]: 100%|██████████| 262/262 [01:38<00:00,  2.67it/s, acc=1, loss=0.00214]    


Epoch 1/15 - Train Loss: 0.0422, Train Acc: 0.9843, Val Loss: 0.0021, Val Acc: 0.9995
Model saved with validation accuracy: 0.9995


Epoch 2/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.93it/s, acc=1, loss=0.000808]


Epoch 2/15 - Train Loss: 0.0125, Train Acc: 0.9955, Val Loss: 0.0008, Val Acc: 0.9996
Model saved with validation accuracy: 0.9996


Epoch 3/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.90it/s, acc=1, loss=0.00165] 


Epoch 3/15 - Train Loss: 0.0076, Train Acc: 0.9977, Val Loss: 0.0017, Val Acc: 0.9995


Epoch 4/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.86it/s, acc=1, loss=0.00138]    


Epoch 4/15 - Train Loss: 0.0070, Train Acc: 0.9975, Val Loss: 0.0014, Val Acc: 0.9998
Model saved with validation accuracy: 0.9998


Epoch 5/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.81it/s, acc=1, loss=0.000935]    


Epoch 5/15 - Train Loss: 0.0084, Train Acc: 0.9974, Val Loss: 0.0009, Val Acc: 0.9995


Epoch 6/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.84it/s, acc=1, loss=0.000152]


Epoch 6/15 - Train Loss: 0.0064, Train Acc: 0.9981, Val Loss: 0.0002, Val Acc: 1.0000
Model saved with validation accuracy: 1.0000


Epoch 7/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.92it/s, acc=1, loss=0.000254]


Epoch 7/15 - Train Loss: 0.0058, Train Acc: 0.9980, Val Loss: 0.0003, Val Acc: 0.9999


Epoch 8/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.75it/s, acc=1, loss=0.000338]


Epoch 8/15 - Train Loss: 0.0041, Train Acc: 0.9986, Val Loss: 0.0003, Val Acc: 0.9999


Epoch 9/15 [Val]: 100%|██████████| 262/262 [00:54<00:00,  4.82it/s, acc=1, loss=9.07e-5] 


Epoch 9/15 - Train Loss: 0.0058, Train Acc: 0.9982, Val Loss: 0.0001, Val Acc: 1.0000


Epoch 10/15 [Val]: 100%|██████████| 262/262 [00:53<00:00,  4.92it/s, acc=1, loss=0.000472]


Epoch 10/15 - Train Loss: 0.0032, Train Acc: 0.9991, Val Loss: 0.0005, Val Acc: 0.9999


Epoch 11/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.94it/s, acc=1, loss=5.03e-5]


Epoch 11/15 - Train Loss: 0.0032, Train Acc: 0.9991, Val Loss: 0.0001, Val Acc: 1.0000


Epoch 12/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.87it/s, acc=1, loss=0.000162]


Epoch 12/15 - Train Loss: 0.0042, Train Acc: 0.9987, Val Loss: 0.0002, Val Acc: 1.0000


Epoch 13/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.87it/s, acc=1, loss=3.28e-5]


Epoch 13/15 - Train Loss: 0.0042, Train Acc: 0.9987, Val Loss: 0.0000, Val Acc: 1.0000


Epoch 14/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.85it/s, acc=1, loss=0.000214]


Epoch 14/15 - Train Loss: 0.0040, Train Acc: 0.9987, Val Loss: 0.0002, Val Acc: 0.9999


Epoch 15/15 [Val]: 100%|██████████| 262/262 [00:33<00:00,  7.93it/s, acc=1, loss=2.06e-5]


Epoch 15/15 - Train Loss: 0.0029, Train Acc: 0.9992, Val Loss: 0.0000, Val Acc: 1.0000
Training complete. Model saved at drowsiness_model.pth
Training history saved to training_history.png


In [1]:
import os
import sys
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import argparse


if __name__ == "__main__":
    
    sys_args = [arg for arg in sys.argv if not arg.startswith('--f=')]
    sys.argv = sys_args


def parse_args():
    parser = argparse.ArgumentParser(description='Train a drowsiness detection model')
    parser.add_argument('--dataset', type=str, default=None,
                        help='Path to the dataset folder containing Drowsy and Non Drowsy subfolders')
    parser.add_argument('--output', type=str, default='drowsiness_model.pth',
                        help='Path to save the trained model')
    parser.add_argument('--batch_size', type=int, default=32, 
                        help='Batch size for training')
    parser.add_argument('--epochs', type=int, default=15,
                        help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate')
    
    
    if 'ipykernel_launcher.py' in sys.argv[0]:
        return parser.parse_args([])
    else:
        return parser.parse_args()

# Custom Dataset class
class DrowsinessDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            if self.transform:
                image = self.transform(image)
            
            label = self.labels[idx]
            return image, label
        except Exception as e:
            print(f"Error loading image {image_path}: {str(e)}")
           
            if self.transform:
                placeholder = torch.zeros((3, 224, 224))
            else:
                placeholder = np.zeros((224, 224, 3), dtype=np.uint8)
            return placeholder, self.labels[idx]

# Data preprocessing
def load_dataset(dataset_path):
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Dataset path does not exist: {dataset_path}")
    
    drowsy_path = os.path.join(dataset_path, "Drowsy")
    non_drowsy_path = os.path.join(dataset_path, "Non Drowsy")
    
    if not os.path.exists(drowsy_path):
        raise FileNotFoundError(f"Drowsy directory not found at: {drowsy_path}")
    if not os.path.exists(non_drowsy_path):
        raise FileNotFoundError(f"Non Drowsy directory not found at: {non_drowsy_path}")
    
    image_paths = []
    labels = []
    
    print(f"Loading drowsy images from: {drowsy_path}")
    # Load drowsy images
    for img_name in os.listdir(drowsy_path):
        if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(drowsy_path, img_name)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(1)  # 1 for drowsy
    
    print(f"Loading non-drowsy images from: {non_drowsy_path}")
    # Load non-drowsy images
    for img_name in os.listdir(non_drowsy_path):
        if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(non_drowsy_path, img_name)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(0)  # 0 for non-drowsy
    
    if len(image_paths) == 0:
        raise ValueError("No valid images found in the dataset directories")
    
    return image_paths, labels

def create_data_loaders(image_paths, labels, batch_size=32, img_size=224):
    # Split the data
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        image_paths, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    # Data transformations
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Create datasets
    train_dataset = DrowsinessDataset(train_paths, train_labels, transform=train_transform)
    val_dataset = DrowsinessDataset(val_paths, val_labels, transform=val_transform)
    
    
    num_workers = 0  
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_loader, val_loader

# Model definition - Using EfficientNet
def create_model():
    model = models.efficientnet_b0(pretrained=True)
    
    # Freeze early layers
    for param in list(model.parameters())[:-10]:
        param.requires_grad = False
    
    # Replace classifier
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(num_features, 2)
    )
    
    return model

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, model_path):
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            train_bar.set_postfix(loss=train_loss/train_total, acc=train_correct/train_total)
        
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = train_correct / train_total
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                val_bar.set_postfix(loss=val_loss/val_total, acc=val_correct/val_total)
        
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        
        scheduler.step(epoch_val_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")
        
        # Save the best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': best_val_acc,
                'model_architecture': 'efficientnet_b0',
            }, model_path)
            print(f"Model saved with validation accuracy: {best_val_acc:.4f}")
    
    return model, history

# Main function
def main():
    # Parse arguments
    args = parse_args()
    

    dataset_path = args.dataset
    if dataset_path is None:
        dataset_path = "Driver Drowsiness Dataset (DDD)"

    
    # Constants
    BATCH_SIZE = args.batch_size
    NUM_EPOCHS = args.epochs
    LEARNING_RATE = args.lr
    IMAGE_SIZE = 224
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MODEL_PATH = args.output
    
    print(f"Using device: {DEVICE}")
    print(f"Dataset path: {dataset_path}")
    print(f"Model will be saved to: {MODEL_PATH}")
    
    try:
        print("Loading dataset...")
        image_paths, labels = load_dataset(dataset_path)
        drowsy_count = labels.count(1)
        non_drowsy_count = labels.count(0)
        print(f"Total images: {len(image_paths)}, Drowsy: {drowsy_count}, Non-Drowsy: {non_drowsy_count}")
        
        print("Creating data loaders...")
        train_loader, val_loader = create_data_loaders(image_paths, labels, BATCH_SIZE, IMAGE_SIZE)
        
        print("Creating model...")
        model = create_model().to(DEVICE)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        print("Starting training...")
        model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                                    scheduler, NUM_EPOCHS, DEVICE, MODEL_PATH)
        
        print(f"Training complete. Model saved at {MODEL_PATH}")
        
        # Plot training history
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Train Accuracy')
        plt.plot(history['val_acc'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        print("Training history saved to training_history.png")
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
    
    

if __name__ == "__main__":
    main()
